# Заметки

# DEV clean

In [22]:
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float, Date, ForeignKey, inspect
from sqlalchemy import text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import datetime
import random
import statistics
from itertools import product
import os

In [2]:
from default_settings import (DWH_USER, 
                              DWH_PASSWORD, 
                              DWH_HOST, 
                              dwh_db_name,
                              api_db_name,
                              source_schema,
                              gen_store_schema,
                              day_gen_store,
                              external_day_gen_store,
                              day_gen_visits_config)

In [3]:
engine_dwh = create_engine(f'postgresql://{DWH_USER}:{DWH_PASSWORD}@{DWH_HOST}:5432/{dwh_db_name}')
engine_api = create_engine(f'postgresql://{DWH_USER}:{DWH_PASSWORD}@{DWH_HOST}:5432/{api_db_name}')

In [4]:
default_params = {'source_tables': {"employers": {"id": {"type": "primary", "data_type": "Integer", "function": "all"}},
                                    "visits": {"line_size": {"type": "extra", "data_type": "Float", "function": "random_range"}},
                                    "product": {"id": {"type": "extra", "data_type": "Integer", "function": "all"}},
                                    "shops": {"id": {"type": "extra", "data_type": "Integer", "function": "all"}} },
                  'extra_columns_length_limit': 500,
                  'gen_records': 1}

In [5]:
target_table_params = {
}

In [6]:
def write_default_params(**kwargs):
    #get variables from kwargs
    source_tables = str(kwargs['source_tables']).replace('\'', '"')
    extra_columns_length_limit = kwargs['extra_columns_length_limit']
    gen_records = kwargs['gen_records']
    current_datetime = datetime.datetime.now()
    
    filling_default_params_query = (f"INSERT INTO {gen_store_schema}.{day_gen_visits_config} "
                                    f"(source_schema, gen_store_schema, day_gen_store, "
                                    f"external_day_gen_store, source_tables, extra_columns_length_limit, gen_records, dt) "
                                    f"VALUES ('{source_schema}', '{gen_store_schema}', '{day_gen_store}', '{external_day_gen_store}', "
                                    f"'{source_tables}', '{extra_columns_length_limit}', '{gen_records}', '{current_datetime}');"
                                   )
    with engine_api.begin() as conn:
        result = conn.execute(text(filling_default_params_query)) 
        conn.commit()

In [7]:
write_default_params(**default_params)

In [8]:
def get_last_param():
    result_dict = {}
    get_params_query = f'SELECT * FROM {gen_store_schema}.{day_gen_visits_config} ORDER BY {day_gen_visits_config}.dt DESC LIMIT 1;'
    with engine_api.connect() as connection:
        result = connection.execute(text(get_params_query))
        column_names = result.keys()
        for row in result:
            for column, row in zip(column_names, row):
                result_dict[column] = row
    return result_dict

In [9]:
current_params = get_last_param()

In [10]:
def create_generated_data_store_table(**kwargs):
    #get variables from kwargs
    day_gen_store = kwargs['day_gen_store']
    gen_store_schema = kwargs['gen_store_schema']
    source_tables = kwargs['source_tables']
    #made sqlalchemy obj
    #create clean metadata
    metadata = MetaData()
    #create inspect
    inspector_db_api = inspect(engine_api)
    
    #generate the store table format from params
    day_gen_store_columns = []
    #add "id" and "date" attribute
    day_gen_store_columns.append(Column(f'id', Integer, primary_key=True))
    day_gen_store_columns.append(Column(f'date', Date))
    #add all attribute from params
    for table_name, columns in source_tables.items():
        for column_name, params in columns.items():
            day_gen_store_columns.append(Column(f'{table_name}_{column_name}', eval(params['data_type'])))
            
    day_gen_store_table = Table(f'{day_gen_store}', 
                                metadata, 
                                *day_gen_store_columns, 
                                schema=f'{gen_store_schema}')

    #checks
    try:
        day_gen_store_inspector = inspector_db_api.get_columns(table_name = day_gen_store,
                                                               schema = gen_store_schema)
        day_gen_store_columns_check = day_gen_store_table.columns.keys()
        day_gen_store_inspector_columns_check = [column['name'] for column in day_gen_store_inspector]
        if set(day_gen_store_columns_check) == set(day_gen_store_inspector_columns_check):
            print(f'table {day_gen_store} without columns changes')
        else:
            print(f'old columns - >> {day_gen_store_inspector_columns_check}')
            print(f'new columns - >> {day_gen_store_columns_check}')
            print(f'table {day_gen_store} columns changes')
            print(f'create all tables from metadata')
            metadata.drop_all(engine_api)
            metadata.create_all(engine_api)
    except:
        print(f'{day_gen_store} сolumn check failed')
        metadata.drop_all(engine_api)
        print(f'table {day_gen_store} not exist')
        print(f'create all tables from metadata')
        metadata.create_all(engine_api)
        
    #return metadata for next steps
    return metadata

In [11]:
metadata = create_generated_data_store_table(**current_params)

day_gen_visits сolumn check failed
table day_gen_visits not exist
create all tables from metadata


In [12]:
def check_filling_generated_data_store_table(metadata, **kwargs):
    #get variables from kwargs
    day_gen_store = kwargs['day_gen_store']
    gen_store_schema = kwargs['gen_store_schema']
    #get current datetime
    current_date = datetime.date.today()
    #metadata from params
    metadata = metadata

    
    #tables from metadata
    day_gen_store_table = metadata.tables.get(day_gen_store)
    
    #check day_gen_store filling
    check_filling_day_gen_store_query = (f"SELECT COUNT(*) FROM {gen_store_schema}.{day_gen_store} "
                                        f"WHERE {gen_store_schema}.{day_gen_store}.date = '{current_date}'"
                                        )
    
    with engine_api.connect() as conn:
        check_filling_day_gen_store_query_result = conn.execute(text(check_filling_day_gen_store_query)).fetchall()
    
    if check_filling_day_gen_store_query_result[0][0] == 0:
        print(f'table {gen_store_schema}.{day_gen_store} does not contain records for the current day.\n'
              f'droping old records and generating new.'
             )
        day_gen_store_empty = True
    else: 
        day_gen_store_empty = False
    return day_gen_store_empty

In [13]:
check_filling_generated_data_store_table(metadata, **current_params)

table api.day_gen_visits does not contain records for the current day.
droping old records and generating new.


True

In [14]:
current_params

{'id': 1,
 'source_schema': 'bd_shops',
 'gen_store_schema': 'api',
 'day_gen_store': 'day_gen_visits',
 'external_day_gen_store': 'external_day_gen_visits',
 'source_tables': {'employers': {'id': {'type': 'primary',
    'data_type': 'Integer',
    'function': 'all'}},
  'visits': {'line_size': {'type': 'extra',
    'data_type': 'Float',
    'function': 'random_range'}},
  'product': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}},
  'shops': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}}},
 'extra_columns_length_limit': 500,
 'gen_records': 1,
 'dt': datetime.datetime(2024, 3, 26, 14, 58, 0, 110787, tzinfo=datetime.timezone.utc)}

In [15]:
def fill_generated_data_store_table(metadata_outside, **kwargs):
    #get variables from kwargs
    source_schema = kwargs['source_schema']
    gen_store_schema = kwargs['gen_store_schema']
    day_gen_store = kwargs['day_gen_store']
    gen_store_schema = kwargs['gen_store_schema']
    source_tables = kwargs['source_tables']
    extra_columns_length_limit = kwargs['extra_columns_length_limit']
    #get current datetime
    current_datetime = datetime.datetime.now()
    ex_tables_dict = {}
    ex_tables_name_dict = {}
    range_columns = {}
    day_gen_store_table = metadata_outside.tables[f'{gen_store_schema}.{day_gen_store}']
    day_gen_store_columns = day_gen_store_table.columns.keys()
    day_gen_store_columns.remove('id')
    day_gen_store_columns = ', '.join(day_gen_store_columns)
    metadata_local = MetaData()
    metadata_local.drop_all(engine_api)
    res_target = (f'INSERT INTO {gen_store_schema}.{day_gen_store} ({day_gen_store_columns}) SELECT CURRENT_DATE as date' )
    res_select = f''
    res_from = f''
    
    for table_name, columns in source_tables.items():
        meta_columns = []
        for column_name, params in columns.items():
            
            if params['function'] == 'random_range':
                with engine_dwh.begin() as conn_source:
                    get_range_query = f'SELECT min({column_name}), max({column_name}) FROM {source_schema}.{table_name};'
                    get_range_query_res = conn_source.execute(text(get_range_query)).fetchall()
                    res_select += f', (FLOOR(RANDOM() * ({get_range_query_res[0][1]} - {get_range_query_res[0][0]} + 1) + {get_range_query_res[0][0]})::numeric(5,2)) as {table_name}_{column_name} '
                conn_source.commit()  
                
            if params['function'] == 'all':
                if params['type'] == 'primary':
                    res_from += f' FROM {gen_store_schema}.ex_{table_name} '
                if params['type'] == 'extra':
                    res_from += f'CROSS JOIN {gen_store_schema}.ex_{table_name} '
                res_select += f', {gen_store_schema}.ex_{table_name}.{table_name}_{column_name} '
                
                meta_columns.append(Column(f'{table_name}_{column_name}', eval(params['data_type'])))
                ex_tables_dict['table'] = Table(f'ex_{table_name}',
                                                metadata_local, 
                                                *meta_columns,
                                                schema=f'{gen_store_schema}')
                metadata_local.create_all(engine_api)
                with engine_dwh.begin() as conn_source:
                    
                    if params['type'] == 'primary':
                        get_values_query = f'SELECT DISTINCT({column_name}) FROM {source_schema}.{table_name};'
                        get_values_query_res = conn_source.execute(text(get_values_query)).fetchall()
                        
                    if params['type'] == 'extra':
                        get_values_query = (f'SELECT ({column_name}) FROM {source_schema}.{table_name} ' 
                                            f'ORDER BY random() LIMIT {extra_columns_length_limit};')
                        get_values_query_res = conn_source.execute(text(get_values_query)).fetchall()
                    conn_source.commit()
            
                with engine_api.begin() as conn_target:
                    insert_values_query = (f'INSERT INTO {gen_store_schema}.ex_{table_name} ({table_name}_{column_name}) ' 
                                           f'VALUES {", ".join([f"({str(i[0])})" for i in get_values_query_res])};')
                    insert_values_query_res = conn_target.execute(text(insert_values_query))
                    
                    conn_target.commit()
   
    with engine_api.begin() as conn_target:
        fill_generated_data_store_table_query = (res_target + res_select + res_from + ';')
        fill_generated_data_store_table_res = conn_target.execute(text(fill_generated_data_store_table_query))
        conn_target.commit()
    metadata_local.drop_all(engine_api)

In [16]:
#fill_generated_data_store_table(metadata, **current_params)

In [17]:
%%time
fill_generated_data_store_table(metadata, **current_params)

CPU times: user 5.82 ms, sys: 13.4 ms, total: 19.2 ms
Wall time: 1min 8s


In [18]:
current_params

{'id': 1,
 'source_schema': 'bd_shops',
 'gen_store_schema': 'api',
 'day_gen_store': 'day_gen_visits',
 'external_day_gen_store': 'external_day_gen_visits',
 'source_tables': {'employers': {'id': {'type': 'primary',
    'data_type': 'Integer',
    'function': 'all'}},
  'visits': {'line_size': {'type': 'extra',
    'data_type': 'Float',
    'function': 'random_range'}},
  'product': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}},
  'shops': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}}},
 'extra_columns_length_limit': 500,
 'gen_records': 1,
 'dt': datetime.datetime(2024, 3, 26, 14, 58, 0, 110787, tzinfo=datetime.timezone.utc)}

In [53]:
def fill_visits_table(**kwargs):
    source_schema = kwargs['source_schema']
    gen_store_schema = kwargs['gen_store_schema']
    day_gen_store = kwargs['day_gen_store']
    gen_store_schema = kwargs['gen_store_schema']
    gen_records = kwargs['gen_records']
    external_day_gen_store = kwargs['external_day_gen_store']
    incr_file_name = f'visits_incr_{datetime.datetime.now().strftime("%Y%m%d_%H%M")}.csv'
    create_ex_table_query = '''
        CREATE TABLE {} AS
            SELECT product_id, 
                   date as visit_date,
                   visits_line_size as line_size,
                   employers_id as employer_id,
                   shops_id as shop_id
            FROM ( 
                SELECT dv.employers_id,
                dv.date, 
                dv.visits_line_size, 
                dv.product_id, 
                dv.shops_id, 
                ROW_NUMBER() OVER (
                                PARTITION BY dv.employers_id 
                                ORDER BY RANDOM()) AS rn 
                FROM api.day_gen_visits dv) AS subquery 
            WHERE rn <= {}) 
    '''
    save_visits_incr_query = '''
        COPY (
            SELECT product_id, 
                   date as visit_date,
                   visits_line_size as line_size,
                   employers_id as employer_id,
                   shops_id as shop_id
            FROM ( 
                SELECT dv.employers_id,
                dv.date, 
                dv.visits_line_size, 
                dv.product_id, 
                dv.shops_id, 
                ROW_NUMBER() OVER (
                                PARTITION BY dv.employers_id 
                                ORDER BY RANDOM()) AS rn 
                FROM api.day_gen_visits dv) AS subquery 
            WHERE rn <= {}) 
        TO '/var/lib/postgresql/GLOBAL_FILE_SHARE/{}' WITH CSV HEADER;
    '''
    save_visits_incr_query = save_visits_incr_query.format(gen_records, incr_file_name)
    
    load_visits_incr_query = '''
        COPY bd_shops.visits (product_id, 
                              visit_date, 
                              line_size, 
                              employer_id, 
                              shop_id)
        FROM '/var/lib/postgresql/GLOBAL_FILE_SHARE/{}'
        DELIMITER ','
        CSV HEADER;
    '''
    load_visits_incr_query = load_visits_incr_query.format(incr_file_name)
    with engine_api.begin() as conn_api:
        save_visits_incr_query_res = conn_api.execute(text(save_visits_incr_query))
        conn_api.commit()
    with engine_dwh.begin() as conn_dwh:
        load_visits_incr_query_res = conn_dwh.execute(text(load_visits_incr_query))
        conn_dwh.commit()
        
    relative_path = "GLOBAL_FILE_SHARE/"
    absolute_path = os.path.join(os.getcwd(), relative_path, incr_file_name)
    if os.path.exists(absolute_path):
        # Delete the file
        os.remove(absolute_path)
        print("File '{}' has been deleted.".format(incr_file_name))
    else:
        print("File '{}' does not exist.".format(incr_file_name))

In [54]:
%%time
fill_visits_table(**current_params)

File 'visits_incr_20240326_1536.csv' has been deleted.
CPU times: user 3.94 ms, sys: 1.09 ms, total: 5.03 ms
Wall time: 12.9 s


File 'visits_incr_20240326_1532.csv' has been deleted.
/home/jovyan/GLOBAL_FILE_SHARE/visits_incr_20240326_1532.csv


In [44]:
absolute_path

'/home/jovyan/GLOBAL_FILE_SHARE/visits_incr_20240326_1530/.csv'

In [42]:
os.getcwd()

'/home/jovyan'

In [36]:
current_params

{'id': 1,
 'source_schema': 'bd_shops',
 'gen_store_schema': 'api',
 'day_gen_store': 'day_gen_visits',
 'external_day_gen_store': 'external_day_gen_visits',
 'source_tables': {'employers': {'id': {'type': 'primary',
    'data_type': 'Integer',
    'function': 'all'}},
  'visits': {'line_size': {'type': 'extra',
    'data_type': 'Float',
    'function': 'random_range'}},
  'product': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}},
  'shops': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}}},
 'extra_columns_length_limit': 500,
 'gen_records': 1,
 'dt': datetime.datetime(2024, 3, 26, 14, 58, 0, 110787, tzinfo=datetime.timezone.utc)}

# Test

In [4]:
import os

relative_path = "path/to/file.txt"
absolute_path = os.path.join(os.getcwd(), relative_path)

In [5]:
absolute_path

'/home/jovyan/path/to/file.txt'

In [ ]:
COPY bd_shops.visits(product_id, visit_date, line_size, employer_id, shop_id)
FROM '/var/lib/postgresql/file_share/test.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
day_gen_store_table = metadata.tables[f'{gen_store_schema}.{day_gen_store}']
day_gen_store_columns = day_gen_store_table.columns.keys()
day_gen_store_columns.remove('id')
day_gen_store_columns = ', '.join(day_gen_store_columns)

res = (f'INSERT INTO {gen_store_schema}.{day_gen_store} ({day_gen_store_columns}) SELECT CURRENT_DATE as date ' 
       f'')
for table_name, columns in source_tables.items():
    for column_name, params in columns.items():
        if 
            res += f'{gen_store_schema}.ex_{table_name}.{table_name}_{column_name} '

print(res)
        
        

In [ ]:
extra_tables = current_params['extra_tables']

In [ ]:
extra_tables 

In [ ]:
for table_name, columns in extra_tables.items():
        for column_name, params in columns.items():
            print(f'{table_name}_{column_name}', (params['data_type']))

In [ ]:
import itertools

In [ ]:
l1 = [(22,), (15,), (19,), (5,), (29,), (4,), (10,), (35,), (6,), (31,), (14,), (13,), (2,), (7,), (38,), (20,), (1,), (18,), (27,), (8,), (11,), (9,), (26,), (30,), (21,), (3,), (17,), (37,), (28,), (34,), (39,), (36,), (16,), (32,), (12,), (24,), (25,), (33,), (23,)]

l1_flat = list(itertools.chain(*l1))

In [ ]:
print(l1[0])

In [ ]:
f'INSERT {l1_flat}'

In [ ]:
l1 = [(22,), (15,), (19,), (5,), (29,), (4,), (10,), (35,), (6,), (31,), (14,), (13,), (2,), (7,), (38,), (20,), (1,), (18,), (27,), (8,), (11,), (9,), (26,), (30,), (21,), (3,), (17,), (37,), (28,), (34,), (39,), (36,), (16,), (32,), (12,), (24,), (25,), (33,), (23,)]

print(f'INSERT INTO `pand` (`id`, `name`) VALUES {", ".join([str(i[0]) for i in l1])};')

In [ ]:
check_test = check_filling_generated_data_store_table(metadata, **default_params)

In [ ]:
check_test['employers_cat_values'][0]

In [ ]:
for i in check_test:
    random.shuffle(check_test[i])

In [ ]:
t = {'employers': {'id': {'type': 'primary',
    'data_type': 'Integer',
    'function': 'all'}},
  'visits': {'line_size': {'type': 'extra',
    'data_type': 'Float',
    'function': 'random_range'}},
  'product': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}},
  'shops': {'id': {'type': 'extra',
    'data_type': 'Integer',
    'function': 'all'}}}


result = ""
for key, value in t.items():
    print(key)
    print(value)

In [ ]:
if value['type'] == 'primary':
        result += f"{key} primary "
    elif value['type'] == 'extra':
        result += f"{key} extra "
print(result.strip())

In [ ]:
default_params['extra_tables']

In [ ]:
for table, column in default_params['extra_tables'].items():
    print(default_params['extra_tables'][table])
    for column in default_params['extra_tables'][table]:
        print(column)

In [ ]:
default_params['extra_tables']['visits']['line_size']['data_type']

In [ ]:
def visit_gen():
    l1 = list(range(1, 55))
    l2 = list(range(1, 44))
    random.shuffle(l1)
    random.shuffle(l2)
    n1 = list(range(1, 6))
    for i,j,n in zip(l1, l2, n1):
        print(str(f'{i} - {j}'))

In [ ]:
l1 = list(range(1, 9999))
l2 = list(range(1, 499))
l3 = list(range(1, 36))
l4 = list(range(1, 499))
random.shuffle(l1)
random.shuffle(l2)
random.shuffle(l3)
random.shuffle(l4)

In [ ]:
zip(l1, l2, l3, l4)
for i,j,n,k in zip(l1, l2, l3,l4):
    print(str(f'{i} - {j} - {n}, {k}'))

In [ ]:
import random

#Define the range and precision
start_range = 0.0 
end_range = 1.0 
precision = 0

#Define the number of values to generate
num_values = 5

#Generate the list of random values
random_values = [round(random.uniform(start_range, end_range), precision) for i in range(num_values)]

print(random_values)

In [ ]:
sql_dict = {'l1': [1, 3, 5, 7],
            'l2': [2, 7, 1],
            'l3': [1, 88, 12, 44, 66, 888]}

In [ ]:
import pandas as pd
from itertools import product

sql_dict = {'l1': [1,3,5,7],
            'l2': [2,7,1],
            'l3': [1,88,12,44,66,888]}

# Create all possible combinations of values from the dictionary
combinations = list(product(*sql_dict.values()))

# Create a DataFrame with the combinations as rows and dictionary keys as columns
df = pd.DataFrame(combinations, columns=sql_dict.keys())

# Print the DataFrame
print(df)

In [ ]:
combinations

In [ ]:
sql_dict['l1']

In [ ]:
import itertools

sql_dict ={'l1': list(range(1, 499)),
           'l2': list(range(1, 9999)),
           'l3': list(range(1, 36))}

# Create a list of all possible combinations of values from the dictionary
combinations = list(itertools.product(*sql_dict.values()))

# Create a table with columns corresponding to the keys of the dictionary
table = {key: [] for key in sql_dict.keys()}

# Populate the table with the combinations of values
for combination in combinations:
    for key, value in zip(sql_dict.keys(), combination):
        table[key].append(value)

# Print the table
for key, values in table.items():
    print(f"{key}: {values}")

In [ ]:
import pandas as pd
import itertools

# Create a sample DataFrame
data = {
    'Column1': [1, 2, 3],
    'Column2': ['A', 'C'],
    'Column3': ['X', 'Y', 'Z']
}

# Get all possible combinations of values
combinations = list(itertools.product(data['Column1'], data['Column2'], data['Column3']))

# Print the result
print(combinations)

In [ ]:
l1 = [(22,), (15,), (19,), (5,), (29,), (4,), (10,), (35,), (6,), (31,), (14,), (13,), (2,), (7,), (38,), (20,), (1,), (18,), (27,), (8,), (11,), (9,), (26,), (30,), (21,), (3,), (17,), (37,), (28,), (34,), (39,), (36,), (16,), (32,), (12,), (24,), (25,), (33,), (23,)]

print(f'{", ".join(f"({str(x[0])})" for x in l1)}')